In [ ]:
import csv
import pandas as pd
import geopy
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS

In [ ]:
restaurant_df = pd.read_csv('raw.csv')
restaurant_df = pd.DataFrame(restaurant_df)

In [ ]:
restaurant_df.head()

In [ ]:
shape = restaurant_df.shape
print (f'The dataframe shape is: {shape}')

In [ ]:
cols = ['state', 'latitude', 'longitude', 'tel', 'price', 'rating']
for col in cols:
    restaurant_df[col] = restaurant_df[col].str.split(': ', expand=True)[1].str.replace("'", '')
restaurant_df.head()

In [ ]:
restaurant_df = restaurant_df.drop(['website', 'country'], axis=1

In [ ]:
coordinates_df = restaurant_df[['latitude', 'longitude']].copy()

In [ ]:
coordinates_df.head()

In [ ]:
gis = GIS("http://www.arcgis.com", "wtcforsyth", "!oV.wnnVpeReETN3UykYsggaMhnpzq1")

def get_zip(coordinates_df, lon_field, lat_field):
    location = reverse_geocode((Geometry({"x":float(coordinates_df[lon_field]), "y":float(coordinates_df[lat_field]), "spatialReference":{"wkid": 4326}})))
    return location['address']['Postal']

zipcodes = coordinates_df.apply(get_zip, axis=1, lat_field='latitude', lon_field='longitude')

In [ ]:
zipcodes.head()

In [ ]:
zipcodes_df = pd.DataFrame(zipcodes, columns=['zipcode'])
zipcodes_df

In [ ]:
restaurant_df = restaurant_df.merge(zipcodes_df.zipcode, left_index=True, right_index=True)
restaurant_df.head()

In [ ]:
column_names = ['restaurant', 'address', 'city', 'state', 'zipcode', 'latitude', 'longitude', 'tel', 'price', 'rating']
restaurant_df = restaurant_df.reindex(columns=column_names)

In [ ]:
restaurant_df.head()

In [ ]:
restaurant_df.to_json('will_data.json')